In [109]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB
from models import *

import types
import copy

In [110]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [111]:
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
# pd.set_option('display.width', 2000)
# pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.max_colwidth', -1)

In [112]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [113]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [114]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [115]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [117]:
# mytrial = []
db = DFDB('../trial/mystack1.3.pkl', auto_commit=False)

In [118]:
db_catboost = DFDB('../trial/catboost.pkl', auto_commit=False)
df_trial_catboost = db_catboost.select()
df_trial_catboost['kfold'] = df_trial_catboost['param'].apply(lambda x: x['kfold'])

In [126]:
df_trial_catboost[['datetime','nfeatures','kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[[1597,1058,1230]]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
1597,2019-05-25 06:38:39.147014,19,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",1.971915,0.002178,2.035012,0.012759,0.063097,tune 794
1058,2019-05-24 13:44:03.330581,19,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",1.985347,0.012792,2.061683,0.626603,0.076336,tune 794
1230,2019-05-24 14:45:46.308242,19,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.903144,0.000020,1.938350,0.000005,0.035207,tune 794 by stratified


In [127]:
db_xgbm = DFDB('../trial/xgbm.pkl', auto_commit=False)
df_trial_xgbm = db_xgbm.select()
df_trial_xgbm['kfold'] = df_trial_xgbm['param'].apply(lambda x: x['kfold'])

In [130]:
df_trial_xgbm.loc[[1591,1646]][['datetime','nfeatures','kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
1591,2019-05-24 17:40:13.932552,12,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",1.982186,0.012094,2.081635,0.536821,0.099449,tune 1444
1646,2019-05-24 18:07:58.010608,12,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.892733,0.000018,1.942439,0.000084,0.049705,tune 1444 by stratified


In [131]:
db_lgbm = DFDB('../trial/lgbm.pkl', auto_commit=False)
df_trial_lgbm = db_lgbm.select()
df_trial_lgbm['kfold'] = df_trial_lgbm['param'].apply(lambda x: x['kfold'])

In [132]:
df_trial_lgbm.loc[[2404,2576]][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
2404,2019-05-23 22:23:36.444552,14,2.008041,0.012310,2.106609,0.548666,0.098568,tune 2367
2576,2019-05-24 02:53:42.468175,14,1.906450,0.000016,1.954530,0.000059,0.048080,tune 2367 by stratified


In [133]:
db_randomforest = DFDB('../trial/randomforest.pkl', auto_commit=False)
df_trial_randomforest = db_randomforest.select()
df_trial_randomforest['kfold'] = df_trial_randomforest['param'].apply(lambda x: x['kfold'])

In [134]:
df_trial_randomforest.loc[[932,1167]][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
932,2019-05-24 15:30:07.425180,3,2.085913,1.413307e-02,2.135108,0.563899,0.049195,tune 724
1167,2019-05-24 16:11:08.433473,3,2.089994,2.707575e-07,2.095076,0.000012,0.005082,tune 724 by stratified


In [135]:
db_extratrees = DFDB('../trial/extratrees.pkl', auto_commit=False)
df_trial_extratrees = db_extratrees.select()
df_trial_extratrees['kfold'] = df_trial_extratrees['param'].apply(lambda x: x['kfold'])

In [136]:
df_trial_extratrees.loc[[1079,1163]][['datetime','nfeatures','kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
1079,2019-05-24 13:21:45.115765,20,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",2.121052,1.425129e-02,2.163110,0.585886,0.042057,tune 946
1163,2019-05-24 15:05:45.525715,20,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",2.126614,2.757785e-07,2.127704,0.000014,0.001090,tune 946 by stratified


In [137]:
db_gradientboosting = DFDB('../trial/gradientboosting.pkl', auto_commit=False)
df_trial_gradientboosting = db_gradientboosting.select()
df_trial_gradientboosting['kfold'] = df_trial_gradientboosting['param'].apply(lambda x: x['kfold'])

In [141]:
df_trial_gradientboosting.loc[[688,835]][['datetime','remark', 'nfeatures', 'kfold', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']]

,datetime,remark,nfeatures,kfold,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
688,2019-05-24 08:14:27.727830,tune 545,20,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",1.974755,0.011996,2.074522,0.535916,0.099766,tune 545
835,2019-05-24 20:03:49.276916,tune 545 by stratified,20,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",1.896027,0.000014,1.943864,0.000147,0.047837,tune 545 by stratified


In [139]:
db_knn = DFDB('../trial/knn.pkl', auto_commit=False)
df_trial_knn = db_knn.select()
df_trial_knn['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [140]:
df_trial_knn.loc[[132,309]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
132,2019-05-25 07:58:22.773360,tune 102,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",15,2.002900,0.012259,2.101092,0.521664,0.098192
309,2019-05-25 17:43:33.288561,tune 102 by stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",15,1.973323,0.000014,1.991212,0.000029,0.017889


In [194]:
db_svr = DFDB('../trial/svr.pkl', auto_commit=False)
df_trial_svr = db_svr.select()
df_trial_svr['kfold'] = df_trial_knn['param'].apply(lambda x: x['kfold'])

In [195]:
df_trial_svr.loc[[122,375]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
122,2019-05-25 19:26:41.288587,tune 87,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",23,1.984271,0.002619,2.070003,0.015321,0.085732
375,2019-05-26 06:53:47.681568,tune 87 by stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",23,1.997429,0.000004,2.004545,0.000004,0.007116


In [177]:
db_fm = DFDB('../trial/fm.pkl', auto_commit=False)
df_trial_fm = db_fm.select()
df_trial_fm['kfold'] = df_trial_fm['param'].apply(lambda x: x['kfold'])

In [178]:
df_trial_fm.loc[[686, 768]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
686,2019-05-26 01:28:33.386373,tune group3,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",20,2.051797,0.003775,2.077350,0.015724,0.025554
768,2019-05-26 01:35:12.392853,tune stratified3,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",20,2.041783,0.000002,2.049726,0.000064,0.007943


In [152]:
db_lasso = DFDB('../trial/lasso.pkl', auto_commit=False)
df_trial_lasso = db_lasso.select()
df_trial_lasso['kfold'] = df_trial_lasso['param'].apply(lambda x: x['kfold'])

In [153]:
df_trial_lasso.loc[[1176,1293]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
1176,2019-05-25 06:16:14.598722,tune 1003,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",31,2.088054,1.346609e-02,2.144718,0.543831,0.056664
1293,2019-05-25 06:27:32.101174,tune 1003 by stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",31,2.108386,1.649935e-07,2.110364,0.000014,0.001978


In [154]:
db_ridge = DFDB('../trial/ridge.pkl', auto_commit=False)
df_trial_ridge = db_ridge.select()
df_trial_ridge['kfold'] = df_trial_ridge['param'].apply(lambda x: x['kfold'])

In [155]:
df_trial_ridge.loc[[251,429]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
251,2019-05-25 06:12:57.901092,tune 199,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",44,2.116238,1.346716e-02,2.167211,0.533201,0.050973
429,2019-05-25 06:17:38.650938,tune 199 by stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",44,2.125651,2.556749e-07,2.127102,0.000027,0.001451


In [175]:
db_kerasmlp = DFDB('../trial/kerasmlp.pkl', auto_commit=False)
df_trial_kerasmlp = db_kerasmlp.select()
df_trial_kerasmlp['kfold'] = df_trial_kerasmlp['param'].apply(lambda x: x['kfold'])

In [176]:
df_trial_kerasmlp[['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].loc[[90,91]]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
90,2019-05-25 23:49:46.507880,None,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",13,2.011389,0.000012,2.015836,0.000033,0.004447
91,2019-05-26 00:19:03.772557,None,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",13,1.976343,0.003094,2.081863,0.014092,0.105520


In [157]:
db_mlp = DFDB('../trial/mlp.pkl', auto_commit=False)
df_trial_mlp = db_mlp.select()
df_trial_mlp['kfold'] = df_trial_mlp['param'].apply(lambda x: x['kfold'])

In [159]:
df_trial_mlp[['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].loc[[18,19]]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
18,2019-05-25 23:51:43.916628,None,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",13,1.903556,0.002639,2.040780,0.011577,0.137224
19,2019-05-25 23:55:11.086624,None,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",13,1.945347,0.000093,1.962482,0.000106,0.017135


In [160]:
db_frgf = DFDB('../trial/frgf.pkl', auto_commit=False)
df_trial_frgf = db_frgf.select()
df_trial_frgf['kfold'] = df_trial_frgf['param'].apply(lambda x: x['kfold'])

In [171]:
df_trial_frgf.loc[[600,522,859]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
600,2019-05-25 13:10:43.881754,tune x group 8,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",20,1.980358,0.011888,2.079444,0.545064,0.099086
522,2019-05-25 10:09:22.575503,tune x group,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",20,1.946860,0.002280,2.046279,0.015523,0.099419
859,2019-05-26 07:14:16.576933,tune x stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",20,1.912713,0.000030,1.956962,0.000047,0.044249


In [181]:
db_decisiontrees = DFDB('../trial/decisiontrees.pkl', auto_commit=False)
df_trial_decisiontrees = db_decisiontrees.select()
df_trial_decisiontrees['kfold'] = df_trial_decisiontrees['param'].apply(lambda x: x['kfold'])

In [182]:
df_trial_decisiontrees.loc[[264,1211]][['datetime','remark','kfold', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,kfold,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
264,2019-05-25 22:46:50.254802,tune by group8,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",12,2.305445,0.014538,2.330928,0.617271,0.025483
1211,2019-05-26 02:09:14.371371,tune by stratified,"{'n_splits': 3, 'random_state': 1985, 'shuffle': True, 'type': 'stratified'}",12,2.097992,0.000156,2.097786,0.000286,-0.000206


In [237]:
#group
# trial_lst = [
#     df_trial_catboost.loc[1597], 
#     df_trial_xgbm.loc[1591], 
#     df_trial_lgbm.loc[2404],
#     df_trial_randomforest.loc[932],
#     df_trial_extratrees.loc[1079],
#     df_trial_gradientboosting.loc[688],
#     df_trial_knn.loc[132], 
#     df_trial_svr.loc[122],
#     df_trial_fm.loc[686],
#     df_trial_lasso.loc[1176], 
#     df_trial_ridge.loc[251],
#     df_trial_kerasmlp.loc[91],
#     df_trial_mlp.loc[18], 
#     df_trial_frgf.loc[522],
#     df_trial_decisiontrees.loc[264]
# ]

name_lst = [
    'catboost', 
    'xgbm', 
    'lgbm',
    'randomforest',
    'extratrees',
    'gradientboosting',
    'knn',
    'svr',
    'fm',
    'lasso',
    'ridge',
    'kerasmlp',
    'mlp',
    'frgf',
    'decisiontrees'
]

# stratified
trial_lst = [
    df_trial_catboost.loc[1230], 
    df_trial_xgbm.loc[1646], 
    df_trial_lgbm.loc[2576],
    df_trial_randomforest.loc[1167],
    df_trial_extratrees.loc[1163],
    df_trial_gradientboosting.loc[835],
    df_trial_knn.loc[309], 
    df_trial_svr.loc[375],
    df_trial_fm.loc[768],
    df_trial_lasso.loc[1293], 
    df_trial_ridge.loc[429],
    df_trial_kerasmlp.loc[90],
    df_trial_mlp.loc[19], 
    df_trial_frgf.loc[859],
    df_trial_decisiontrees.loc[1211]
]

# name_lst = [
#     'catboost', 
#     'xgbm', 
#     'lgbm',
#     'randomforest',
#     'extratrees',
#     'gradientboosting',
#     'knn',
#     'svr',
#     'fm',
#     'lasso',
#     'ridge',
#     'kerasmlp',
#     'mlp',
#     'frgf',
#     'decisiontrees'
# ]

catboost_columns = ['spkt_welch_density__coeff_3',
 'spkt_welch_densitycoeff_2',
 'abs_q25_5',
 'abs_q75_6',
 'q05_roll_std_1000',
 'abs_q75_7',
 'abs_q95_2',
 'q05_5',
 'abs_q75_2',
 '5000skewness_max_',
 'fft_coefficientcoeff_80__attr_"imag"',
 "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
 '5000kurtosis_mean_',
 "number_peaks{'n': 1}",
 '5000smoothness_entropy_',
 'ave10_7',
 'q75_roll_std_1000',
 'FFT_Mag_25q0',
 'fft_coefficientcoeff_20__attr_"abs"']

df_train_stacknet = pd.DataFrame()
df_test_stacknet = pd.DataFrame()
for df_, name_ in zip(trial_lst, name_lst):
    try:
        df_test_stacknet_i = df_['df_test_pred']
        df_test_stacknet[name_] = np.mean(df_test_stacknet_i.drop(columns=['index']).values, axis=1)
        
        df_train_stacknet_i = df_['df_valid_pred']
        df_train_stacknet[name_] = df_train_stacknet_i['predict']
        
    except Exception as e:
        print(name_, 'exception')

df_train_stacknet['index'] = df_train_stacknet_i['index']
df_train_stacknet = pd.merge(df_train_stacknet, df_train[['y','index', 'group','label']+original_columns], on='index')
df_test_stacknet['index'] = df_test_stacknet_i['index']
df_test_stacknet = pd.merge(df_test_stacknet, df_test[['index']+original_columns], on='index')

In [197]:
mytrial=[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1e2)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_loguniform('random_strength', 1e-3, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':name_lst+catboost_columns,
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
                'logging_level': 'Silent'
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train_stacknet, args, df_test = df_test_stacknet, trial=mytrial, remark='25 tune catboost_columns stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=400)

[I 2019-05-26 05:04:44,201] Finished a trial resulted in value: 0.07627917455593128. Current best value is 0.07627917455593128 with parameters: {'num_trees': 428, 'depth': 6, 'learning_rate': 0.01985675937953895, 'l2_leaf_reg': 0.08919649668210289, 'bagging_temperature': 0.8439297495153497, 'random_strength': 0.28269689717595414, 'random_state': 3292}.
[I 2019-05-26 05:07:25,934] Finished a trial resulted in value: 0.5035806432468859. Current best value is 0.07627917455593128 with parameters: {'num_trees': 428, 'depth': 6, 'learning_rate': 0.01985675937953895, 'l2_leaf_reg': 0.08919649668210289, 'bagging_temperature': 0.8439297495153497, 'random_strength': 0.28269689717595414, 'random_state': 3292}.
[I 2019-05-26 05:08:19,182] Finished a trial resulted in value: 0.3103294589147875. Current best value is 0.07627917455593128 with parameters: {'num_trees': 428, 'depth': 6, 'learning_rate': 0.01985675937953895, 'l2_leaf_reg': 0.08919649668210289, 'bagging_temperature': 0.8439297495153497, 

[I 2019-05-26 05:15:41,144] Finished a trial resulted in value: 0.10783147215956115. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.998338370264902, 'random_strength': 0.0010766553936203042, 'random_state': 5835}.
[I 2019-05-26 05:15:47,122] Finished a trial resulted in value: 0.1174078724230861. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.998338370264902, 'random_strength': 0.0010766553936203042, 'random_state': 5835}.
[I 2019-05-26 05:16:10,217] Finished a trial resulted in value: 0.3444953775689802. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.9983

[I 2019-05-26 05:23:51,435] Finished a trial resulted in value: 0.23748784965112668. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.998338370264902, 'random_strength': 0.0010766553936203042, 'random_state': 5835}.
[I 2019-05-26 05:24:29,861] Finished a trial resulted in value: 0.2429116623482746. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.998338370264902, 'random_strength': 0.0010766553936203042, 'random_state': 5835}.
[I 2019-05-26 05:24:39,736] Finished a trial resulted in value: 0.3786509236572709. Current best value is 0.022883504867524437 with parameters: {'num_trees': 217, 'depth': 4, 'learning_rate': 0.011609162391093593, 'l2_leaf_reg': 0.0010216184929788175, 'bagging_temperature': 0.9983

[I 2019-05-26 05:33:02,885] Finished a trial resulted in value: 0.06591563678346328. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989, 'random_strength': 0.011511549608092768, 'random_state': 6927}.
[I 2019-05-26 05:33:07,117] Finished a trial resulted in value: 0.04351913042320459. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989, 'random_strength': 0.011511549608092768, 'random_state': 6927}.
[I 2019-05-26 05:33:11,289] Finished a trial resulted in value: 0.024371034240656498. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989

[I 2019-05-26 05:36:15,159] Finished a trial resulted in value: 0.1651563264484491. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989, 'random_strength': 0.011511549608092768, 'random_state': 6927}.
[I 2019-05-26 05:36:19,671] Finished a trial resulted in value: 0.047721667847363064. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989, 'random_strength': 0.011511549608092768, 'random_state': 6927}.
[I 2019-05-26 05:36:23,957] Finished a trial resulted in value: 0.10202033627507032. Current best value is 0.022800017484797288 with parameters: {'num_trees': 355, 'depth': 3, 'learning_rate': 0.01158506676569636, 'l2_leaf_reg': 39.37679746039934, 'bagging_temperature': 0.9113741783164989,

...

[I 2019-05-26 06:54:17,903] Finished a trial resulted in value: 0.020822590384228845. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 06:54:24,450] Finished a trial resulted in value: 0.09564857680818827. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 06:54:29,133] Finished a trial resulted in value: 0.017817396345768413. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738

[I 2019-05-26 07:00:20,335] Finished a trial resulted in value: 0.062164493794491876. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 07:00:27,833] Finished a trial resulted in value: 0.12320516367644224. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 07:00:32,055] Finished a trial resulted in value: 0.059668956592639585. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738

[I 2019-05-26 07:02:51,356] Finished a trial resulted in value: 0.13856305547054884. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 07:02:55,276] Finished a trial resulted in value: 0.02515257883666209. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288, 'random_strength': 0.3240617778559526, 'random_state': 2528}.
[I 2019-05-26 07:03:15,050] Finished a trial resulted in value: 0.1459210137879573. Current best value is 0.01055959813956765 with parameters: {'num_trees': 205, 'depth': 2, 'learning_rate': 0.01123870871257948, 'l2_leaf_reg': 0.012368517126939186, 'bagging_temperature': 0.8014857443738288

[I 2019-05-26 07:05:25,167] Finished a trial resulted in value: 0.058868797721379366. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.8697253796442558, 'random_strength': 0.5822983157342466, 'random_state': 834}.
[I 2019-05-26 07:05:29,369] Finished a trial resulted in value: 0.026746125529143818. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.8697253796442558, 'random_strength': 0.5822983157342466, 'random_state': 834}.
[I 2019-05-26 07:05:32,980] Finished a trial resulted in value: 0.012266260608672133. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.86972537

[I 2019-05-26 07:07:24,230] Finished a trial resulted in value: 0.07255289860535596. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.8697253796442558, 'random_strength': 0.5822983157342466, 'random_state': 834}.
[I 2019-05-26 07:07:36,343] Finished a trial resulted in value: 0.05147840279284586. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.8697253796442558, 'random_strength': 0.5822983157342466, 'random_state': 834}.
[I 2019-05-26 07:07:49,296] Finished a trial resulted in value: 0.09919792881874469. Current best value is 0.005875607731970951 with parameters: {'num_trees': 200, 'depth': 2, 'learning_rate': 0.010016713392895902, 'l2_leaf_reg': 0.012788770906704305, 'bagging_temperature': 0.86972537964

In [238]:
# def remove_useless_columns(df_train, param, df_test, trial, score, remark=None):
#     original_columns = copy.deepcopy(param['columns'])
#     for col in original_columns:
#         args = copy.deepcopy(param)
#         args['columns'] = list(set(original_columns)-set([col]))
#         df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test, trial, remark)
#         val_mae_mean = np.mean(df_his.valid)
#         if val_mae_mean<score:
#             remove_useless_columns(df_train, args, df_test, trial, val_mae_mean, remark)
#     return

In [279]:
# mytrial=[]
args=copy.deepcopy(df_trial.loc[642]['param'])
args['columns'] = [
    'catboost',
 'xgbm',
 'lgbm',
 'randomforest',
 'extratrees',
 'gradientboosting',
 'knn',
 'svr',
 'fm',
 'lasso',
 'ridge',
 'kerasmlp',
 'mlp',
 'frgf',
#  'decisiontrees',
 '5000kurtosis_mean_',
 'q05_5',
 'spkt_welch_densitycoeff_2',
 'abs_q75_2',
 'abs_q75_6',
 'ave10_7',
 "number_peaks{'n': 1}",
 'abs_q25_5',
 '5000smoothness_entropy_',
 'spkt_welch_density__coeff_3',
 'FFT_Mag_25q0',
 'abs_q95_2',
 'q75_roll_std_1000',
 'fft_coefficientcoeff_20__attr_"abs"',
#  '5000skewness_max_',
 'q05_roll_std_1000',
 'fft_coefficientcoeff_80__attr_"imag"'
]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train_stacknet, args, df_test = df_test_stacknet, trial=mytrial, remark='tune 493 2')

In [280]:
pd.DataFrame(mytrial)[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-26 08:11:32.853580,tune 493 2,34,1.811131,0.000009,1.856968,0.000133,0.045837
1,2019-05-26 08:12:30.952589,tune 493 2,33,1.813043,0.000014,1.856637,0.000083,0.043595
2,2019-05-26 08:17:49.456323,tune 493 2,31,1.813436,0.000006,1.855016,0.000108,0.041580
3,2019-05-26 08:18:56.152059,tune 493 2,30,1.815448,0.000019,1.855647,0.000120,0.040198
4,2019-05-26 08:19:28.153232,tune 493 2,30,1.814150,0.000007,1.854725,0.000112,0.040575
5,2019-05-26 08:20:02.804103,tune 493 2,29,1.815807,0.000014,1.855737,0.000119,0.039930
6,2019-05-26 08:20:36.019321,tune 493 2,29,1.815354,0.000014,1.855166,0.000113,0.039812
7,2019-05-26 08:31:36.789412,tune 493 2,30,1.769426,0.000008,1.842754,0.000139,0.073328
8,2019-05-26 08:37:00.988210,tune 493 2,30,1.762529,0.000044,1.838039,0.000081,0.075510


In [271]:
for trial_i in mytrial[-1:]:
    db.insert(trial_i)
db.commit()
df_trial = db.select()

In [281]:
df_trial[(df_trial['mae_diff']<.05)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae']).head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
807,2019-05-26 08:19:28.153232,tune 493 2,30,1.814150,0.000007,1.854725,0.000112,0.040575
801,2019-05-26 07:54:02.000427,tune 493,29,1.814039,0.000013,1.854749,0.000120,0.040710
805,2019-05-26 08:17:49.456323,tune 493 2,31,1.813436,0.000006,1.855016,0.000108,0.041580
809,2019-05-26 08:20:36.019321,tune 493 2,29,1.815354,0.000014,1.855166,0.000113,0.039812
806,2019-05-26 08:18:56.152059,tune 493 2,30,1.815448,0.000019,1.855647,0.000120,0.040198
808,2019-05-26 08:20:02.804103,tune 493 2,29,1.815807,0.000014,1.855737,0.000119,0.039930
804,2019-05-26 08:12:30.952589,tune 493 2,33,1.813043,0.000014,1.856637,0.000083,0.043595
493,2019-05-26 05:37:29.837931,25 tune catboost_columns stratified,34,1.811131,0.000009,1.856968,0.000133,0.045837
803,2019-05-26 08:11:32.853580,tune 493 2,34,1.811131,0.000009,1.856968,0.000133,0.045837
800,2019-05-26 07:53:03.030664,tune 493,26,1.820090,0.000007,1.860003,0.000075,0.039914


In [276]:
df_trial[(df_trial['mae_diff']<.05)][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].sort_values(by=['val_mae']).head(1)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
807,2019-05-26 08:19:28.153232,tune 493 2,30,1.81415,0.000007,1.854725,0.000112,0.040575


In [275]:
idx = 807
df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['seg_id'] = df_test_pred['index']
df_submit.to_csv('submission_mystacknet_{}.csv'.format(idx), index=False)

In [225]:
db.commit()

In [288]:
db2 = DFDB('../trial/mystack.pkl', auto_commit=False)
df_trial2 = db2.select()
df_trial2.loc[[562]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
562,2019-05-19 20:38:58.981647,"add mlp,rgf tune stratified",43,1.734277,0.000014,1.784253,0.000267,0.049976
841,2019-05-25 09:20:34.138991,tune 562,33,1.317244,0.000871,1.690598,0.000037,0.373355
